In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    database_name = '%s_upcache' % game
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{database_name}', connect_args={'connect_timeout': 10})

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1640217600
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_game_content_okr_spend_supply_sql(game, start_time, end_time):
    return """SELECT cur.time+1*86400 AS `day`,
              SUM(cur.total_amount) AS current_qtr_supply,
              (SELECT SUM(prev.total_amount)
              FROM `tr_active_player_resource_levels` AS prev
              WHERE (prev.time + 90*86400) BETWEEN cur.time - 43200 AND cur.time + 43200
              AND prev.townhall_level >= 5
              AND prev.country_tier IN ('1','2')
              AND prev.resource = 'gamebucks') as previous_qtr_supply
              FROM `tr_active_player_resource_levels` AS cur
              WHERE cur.time >= %d
              AND cur.time < %d + 86400
              AND cur.townhall_level >= 5
              AND cur.country_tier IN ('1','2')
              AND cur.resource = 'gamebucks'
              GROUP BY `day`
              ORDER BY `day` ASC LIMIT 1000;
              """ % (start_time, end_time)

def get_maintenance_okr_ccl_dau_sql(game, start_time, end_time, level):
    return """SELECT main.day AS `day`,
              (SELECT SUM(prev.dau)
              FROM %s_sessions_daily_summary prev
              WHERE prev.day = main.day - 90*86400
              AND prev.townhall_level >= %d) AS `prevqtr`,
              SUM(main.dau) AS `thisqtr`
              FROM %s_sessions_daily_summary main
              WHERE main.day >= %d AND main.day < %d
              AND main.townhall_level >= %d
              GROUP BY `day` ORDER BY `day` ASC;""" % (game, level, game, start_time, end_time, level)

def get_maintenance_okr_receipts_sql(game, start_time, end_time, level):
    return """SELECT cred.day + 1*86400 AS `day`,
              SUM(cred.usd_receipts_cents)/100 AS current_qtr_usd,
              (SELECT SUM(prev.usd_receipts_cents)/100
              FROM `%s_credits_daily_summary` AS prev
              WHERE prev.day = cred.day - 90*86400
              AND prev.townhall_level >= %d) as previous_qtr_usd
              FROM `tr_credits_daily_summary` AS cred
              WHERE cred.day >= %d
              AND cred.day < %d
              AND cred.townhall_level >= %d
              GROUP BY `day`
              ORDER BY `day` ASC LIMIT 1000;""" % (game, level, start_time, end_time, level)

def get_maintenance_okr_receipts_per_spend_sql(game, start_time, end_time, level):
    return """SELECT timeline.day AS `day`,
              (SELECT SUM(rolling.usd_receipts_cents)
              FROM `%s_credits_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 AND rolling.day <= timeline.day
              AND rolling.townhall_level >= %d) AS `dollars`,
              (SELECT SUM(rolling.total_price)
              FROM `%s_store_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 AND rolling.day <= timeline.day
              AND rolling.townhall_level >= %d
              AND rolling.currency = 'gamebucks') AS `gamebucks`,
              (SELECT SUM(rolling.usd_receipts_cents)
              FROM `%s_credits_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 - 90*86400 AND rolling.day <= timeline.day - 90*86400
              AND rolling.townhall_level >= %d) AS `dollarsprev`,
              (SELECT SUM(rolling.total_price)
              FROM `%s_store_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 - 90*86400 AND rolling.day <= timeline.day - 90*86400
              AND rolling.townhall_level >= %d
              AND rolling.currency = 'gamebucks') AS `gamebucksprevqtr`
              FROM (SELECT sess.day AS day
              FROM `%s_sessions_daily_summary` AS sess
              WHERE sess.day >= %d AND sess.day < %d
              GROUP BY sess.day ORDER BY sess.day ASC) AS timeline LIMIT 999999;""" % (game, level, game, level, game, level, game, level, game, start_time, end_time)

def get_maintenance_okr_gold_stock_avg_sql(game, start_time, end_time, level):
    return """SELECT 86400*ROUND(time/86400) AS `day`,
              IF(SUM(num_players)>0,SUM(total_amount)/SUM(num_players),NULL) AS `average`
              FROM %s_active_player_resource_levels
              WHERE resource = 'gamebucks'
              AND time >= %d AND time < %d
              AND country_tier IN ('1' , '2')
              AND townhall_level >= %d
              GROUP BY `Day` ORDER BY `Day` ASC LIMIT 1000;""" % (game, start_time, end_time, level)

def get_maintenance_okr_toc_promotion_L2_sql(game, start_time, end_time, frame_mod):
    return """SELECT timeline.day AS `day`,
              SUM(IF(th2.user_id IS NOT NULL,1,0)) / SUM(1) AS `promotion`
              FROM (SELECT sess.day AS DAY
              FROM `%s_sessions_daily_summary` AS sess
              WHERE sess.day >= %d-90*86400
              AND sess.day < %d
              GROUP BY sess.day ORDER BY sess.day ASC) AS timeline,
              (SELECT user_id, account_creation_time
              FROM `%s_upcache` AS up_raw
              WHERE up_raw.account_creation_time + 7*86400 >= %d - 90*86400
              AND up_raw.account_creation_time + 1*86400 < %d
              AND frame_platform %s) AS temp_up
              LEFT JOIN tr_townhall_at_time th2 ON (th2.user_id = temp_up.user_id
                                                    AND th2.townhall_level = 2 -- threshold TOC level
                                                    AND th2.time >= temp_up.account_creation_time
                                                    AND th2.time < temp_up.account_creation_time+1*86400)
              WHERE temp_up.account_creation_time >= timeline.day - 7*86400
              AND temp_up.account_creation_time < timeline.day - 1*86400
              GROUP BY timeline.day
              ORDER BY timeline.day ASC;""" % (game, start_time, end_time, game, start_time, end_time, frame_mod)

def get_maintenance_okr_toc_promotion_L3_sql(game, start_time, end_time, frame_mod):
    return """SELECT timeline.day AS `day`,
              (SELECT SUM(IF(EXISTS(SELECT 1
                                    FROM %s_townhall_at_time th2
                                    WHERE th2.user_id = th.user_id
                                    AND th2.townhall_level = 3
                                    AND th2.time <= th.time+10*86400),1,0)) / COUNT(1)
              FROM %s_townhall_at_time th INNER JOIN %s_upcache u
              WHERE th.townhall_level = 2
              AND th.time >= timeline.day - 20*86400
              AND th.time < timeline.day - 10*86400
              AND u.user_id = th.user_id
              AND u.frame_platform %s) AS `promotion`
              FROM (SELECT sess.day AS day
              FROM `%s_sessions_daily_summary` AS sess
              WHERE sess.day >= %d - 90*86400
              AND sess.day < %d
              GROUP BY sess.day ORDER BY sess.day ASC) AS timeline;""" % (game, game, game, frame_mod, game, start_time, end_time)

time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

# prepare maintenance OKR data

engine = get_engine('tr')
maintenance_okr_ccl_dau_sql = get_maintenance_okr_ccl_dau_sql('tr',start_time,end_time,5)
maintenance_okr_ccl_dau_data = pd.read_sql(maintenance_okr_ccl_dau_sql, engine)
maintenance_okr_receipts_sql = get_maintenance_okr_receipts_sql('tr',start_time,end_time,5)
maintenance_okr_receipts_data  = pd.read_sql(maintenance_okr_receipts_sql, engine)
maintenance_okr_receipts_per_spend_sql = get_maintenance_okr_receipts_per_spend_sql('tr',start_time,end_time,5)
maintenance_okr_receipts_per_spend_data  = pd.read_sql(maintenance_okr_receipts_per_spend_sql, engine)
maintenance_okr_gold_stock_avg_sql = get_maintenance_okr_gold_stock_avg_sql('tr', start_time, end_time, 5)
maintenance_okr_gold_stock_avg_data  = pd.read_sql(maintenance_okr_gold_stock_avg_sql, engine)
maintenance_okr_toc_promotion_L2_sql = get_maintenance_okr_toc_promotion_L2_sql('tr', start_time, end_time, '!= "bh"')
maintenance_okr_toc_promotion_L2_data = pd.read_sql(maintenance_okr_toc_promotion_L2_sql, engine)
maintenance_okr_toc_promotion_L2_electron_sql = get_maintenance_okr_toc_promotion_L2_sql('tr', start_time, end_time, '= "bh"')
maintenance_okr_toc_promotion_L2_electron_data = pd.read_sql(maintenance_okr_toc_promotion_L2_electron_sql, engine)
maintenance_okr_toc_promotion_L3_sql = get_maintenance_okr_toc_promotion_L3_sql('tr', start_time, end_time, '!= "bh"')
maintenance_okr_toc_promotion_L3_data = pd.read_sql(maintenance_okr_toc_promotion_L3_sql, engine)
maintenance_okr_toc_promotion_L3_electron_sql = get_maintenance_okr_toc_promotion_L3_sql('tr', start_time, end_time, '= "bh"')
maintenance_okr_toc_promotion_L3_electron_data = pd.read_sql(maintenance_okr_toc_promotion_L3_electron_sql, engine)

print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

In [2]:
# maintenance OKR display
boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries['start'] = diplay_boundaries['start'] + 86400*7
print('TR CCL5+ DAU vs. previous quarter end')
header = "{0:12} {1:8}".format('Week', 'DAU')
print(header)
for i, day in enumerate(maintenance_okr_ccl_dau_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    maintenance_okr_ccl_dau = maintenance_okr_ccl_dau_data['thisqtr'][i]
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(int(maintenance_okr_ccl_dau)))
        print(output_line)
print('')
print('TR Monetization: Cumulative Sum of Receipts per CCL5+ DAU')
header = "{0:12} {1:8}".format('Week', 'Receipts/DAU')
print(header)
maintenance_okr_ccl_receipts = 0.0
for i, day in enumerate(maintenance_okr_receipts_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    maintenance_okr_ccl_dau = maintenance_okr_ccl_dau_data['thisqtr'][i]
    maintenance_okr_ccl_receipts += maintenance_okr_receipts_data['current_qtr_usd'][i] / maintenance_okr_ccl_dau
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(round(maintenance_okr_ccl_receipts, 2)))
        print(output_line)
print('')
print('TR USD Receipts per gamebuck')
header = "{0:12} {1:8}".format('Week', 'Receipts/Gamebuck')
print(header)
for i, day in enumerate(maintenance_okr_receipts_per_spend_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_receipts = maintenance_okr_receipts_per_spend_data['dollars'][i] / maintenance_okr_receipts_per_spend_data['gamebucks'][i]
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(round(output_receipts, 2)))
        print(output_line)
print('')
print('TR Gold Stock Average CCL5+ Tier 1/2 vs. previous quarter')
header = "{0:12} {1:8}".format('Week', 'Average Gold Stock')
print(header)
for i, day in enumerate(maintenance_okr_receipts_per_spend_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(int(maintenance_okr_gold_stock_avg_data['average'][i])))
        print(output_line)

print('')
print('TR TOC L2 promotion within 1d (non-Electron)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L2_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L2_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)

print('')
print('TR TOC L2->L3 promotion within 10d (non-Electron)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L3_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L3_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)

print('')
print('TR TOC L2 promotion within 1d (Electron only)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L2_electron_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L2_electron_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)

print('')
print('TR TOC L2->L3 promotion within 10d (Electron)')
header = "{0:12} {1:8}".format('Week', 'Promotion Percentage')
print(header)
for i, day in enumerate(maintenance_okr_toc_promotion_L3_electron_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_percentage = '{0:.0%}'.format(maintenance_okr_toc_promotion_L3_electron_data['promotion'][i])
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, output_percentage)
        print(output_line)